## Load data

In [ ]:
import sys
 
sys.path.insert(0, "../../")

In [ ]:
from dataAnalysis.data.Filter import Filter
import pandas as pd
from dataAnalysis.Constants import *
from dataAnalysis.DataAnalysis import DataAnalysis

FEATURES.extend(['Noise0', 'Noise1', 'Noise2', 'Noise3', 'Noise4', 'Noise5', 'Noise6', 'Noise7', 'Noise8', 'Noise9', 'Noise10', 'Noise11', 'Noise12', 'Noise13', 'Noise14', 'Noise15', 'Noise16', 'Noise17', 'Noise18', 'Noise19', 'Noise20', 'Noise21', 'Noise22', 'Noise23', 'Noise24', 'Noise25', 'Noise26', 'Noise27', 'Noise28', 'Noise29', 'Noise30', 'Noise31', 'Noise32', 'Noise33', 'Noise34', 'Noise35', 'Noise36', 'Noise37', 'Noise38', 'Noise39', 'Noise40', 'Noise41', 'Noise42', 'Noise43', 'Noise44', 'Noise45', 'Noise46', 'Noise47', 'Noise48', 'Noise49', 'Noise50', 'Noise51', 'Noise52', 'Noise53', 'Noise54', 'Noise55', 'Noise56', 'Noise57', 'Noise58', 'Noise59', 'Noise60', 'Noise61', 'Noise62', 'Noise63', 'Noise64', 'Noise65', 'Noise66', 'Noise67', 'Noise68', 'Noise69', 'Noise70', 'Noise71', 'Noise72', 'Noise73', 'Noise74', 'Noise75', 'Noise76', 'Noise77', 'Noise78', 'Noise79', 'Noise80', 'Noise81', 'Noise82', 'Noise83', 'Noise84', 'Noise85', 'Noise86', 'Noise87', 'Noise88', 'Noise89', 'Noise90', 'Noise91', 'Noise92', 'Noise93', 'Noise94', 'Noise95', 'Noise96', 'Noise97', 'Noise98', 'Noise99'])
data = pd.read_csv(r"../../extdata/noisy_sbc_100.csv", header=0)
data_analysis = DataAnalysis(data)

In [ ]:
import torch

y_train = torch.tensor(data_analysis.get_y_train(), dtype=torch.long)
X_train = torch.tensor(data_analysis.get_X_train(), dtype=torch.float)

y_test = torch.tensor(data_analysis.get_y_test(), dtype=torch.long)
X_test = torch.tensor(data_analysis.get_X_test(), dtype=torch.float)

y_gw_test = torch.tensor(data_analysis.get_y_gw(), dtype=torch.long)
X_gw_test = torch.tensor(data_analysis.get_X_gw(), dtype=torch.float)

In [ ]:
y_all = torch.concat((y_train, y_test, y_gw_test))
X_all = torch.concat((X_train, X_test, X_gw_test))

## Train/Validation/Test splits

In [ ]:
def true_indices_like(tensor):
    return torch.ones((tensor.shape[0])).type(torch.bool)

def false_indices_like(tensor):
    return torch.zeros((tensor.shape[0])).type(torch.bool)

def split(train_features):
    tensor = true_indices_like(train_features)
    max_index = round(tensor.shape[0] * 0.8)
    train = torch.zeros(tensor.shape[0])
    train[:max_index] = 1
    
    val = torch.zeros(tensor.shape[0])
    val[max_index:] = 1
    return{
        "train": train.type(torch.bool),
        "val":val.type(torch.bool)
    }
train_data = split(X_train)

train_mask = torch.concat((train_data["train"], false_indices_like(X_test), false_indices_like(X_gw_test)))
val_mask = torch.concat((train_data["val"], false_indices_like(X_test), false_indices_like(X_gw_test)))
test_l_mask = torch.concat((false_indices_like(X_train), true_indices_like(X_test), false_indices_like(X_gw_test)))
test_gw_mask = torch.concat((false_indices_like(X_train), false_indices_like(X_test), true_indices_like(X_gw_test)))

## Graph construction

In [ ]:
import torch
import numpy as np
from dataAnalysis.Constants import *

def to_tensor(df):
    return torch.Tensor(list(df.values))

def get_quantil_tensor():
    number_of_quantiles = 10
    q = torch.arange(0, 1, 1/number_of_quantiles)
    q = torch.Tensor([0.025,0.05, 0.1, 0.2, 0.35, 0.5, 0.65, 0.8, 0.9, 0.95, 0.975, 1])
    return q

def get_quantiles(tensor):
    q = get_quantil_tensor() 
    return torch.quantile(tensor, q)

def normalize(tensor):
    mean = torch.mean(tensor, dim = 0)
    std = torch.std(tensor, dim = 0)
    mean_diff = tensor - mean
    return mean_diff / std

def get_quantile_indices(tensor, quantiles):
    quantile_indices = []
    all_indices = torch.Tensor([])
    prev_quantile = -1e-4
    indices_control = torch.arange(0, tensor.shape[0])
    for i in range(quantiles.nelement()):
        indices_u = (tensor > prev_quantile).nonzero(as_tuple=True)[0] # (tensor > prev_quantile and tensor <= quantiles[i]).nonzero(as_tuple=True)[0]
        indices_o = (tensor <= quantiles[i]).nonzero(as_tuple=True)[0]
        indices = torch.from_numpy(np.intersect1d(indices_u, indices_o))
        quantile_indices.append(indices)
        prev_quantile = quantiles[i]
    return quantile_indices


def create_node_features(node_type, quantiles):
    nodes_features = []
    prev_quantile = torch.Tensor([0])
    for i in range(quantiles.nelement()):
        node_features = [prev_quantile.item(), quantiles[i].item(), get_quantil_tensor()[i].item()]
        prev_quantile = quantiles[i]
        nodes_features.append(node_features)
    return torch.tensor(nodes_features)

def create_edge_features_to_patient(node_type, quantile_indices):
    source_edge_list = None
    target_edge_list = None
    for i in range(len(quantile_indices)):
        target_edges = torch.ones((quantile_indices[i].nelement())) * i
        source_edges = quantile_indices[i]
        source_edge_list = source_edges if source_edge_list is None else torch.concat((source_edge_list, source_edges))
        target_edge_list = target_edges if target_edge_list is None else torch.concat((target_edge_list, target_edges))
    return torch.stack([source_edge_list, target_edge_list]).type(torch.long)

def add_features_and_edges(graph):
    for i, feature_name in enumerate(FEATURES):
        if feature_name not in [HGB_COLUMN_NAME, WBC_COLUMN_NAME, RBC_COLUMN_NAME, MCV_COLUMN_NAME, PLT_COLUMN_NAME]:
            continue
        feature_vector = graph[PATIENT_NAME].x[:, i]
        node_quantiles = get_quantiles(feature_vector)
        quantile_indices = get_quantile_indices(feature_vector, node_quantiles)
        graph[feature_name].x = create_node_features(feature_name, node_quantiles)
        graph[PATIENT_NAME, EDGE_TYPE, feature_name].edge_index = create_edge_features_to_patient(feature_name, quantile_indices)

In [ ]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from imblearn.under_sampling import RandomUnderSampler

def normalize(tensor):
    mean = torch.mean(tensor, dim = 0)
    std = torch.std(tensor, dim = 0)
    mean_diff = tensor - mean
    return mean_diff / std

graph = HeteroData()
graph[PATIENT_NAME].x = X_all
add_features_and_edges(graph)
graph[PATIENT_NAME].y = y_all
graph[PATIENT_NAME].train_mask = train_mask
graph[PATIENT_NAME].val_mask = val_mask
graph[PATIENT_NAME].test_l_mask = test_l_mask
graph[PATIENT_NAME].test_gw_mask = test_gw_mask
graph = T.ToUndirected()(graph)
graph[PATIENT_NAME].x = normalize(graph[PATIENT_NAME].x)

In [ ]:
graph

### Model defintion

In [ ]:
from torch_geometric.nn import GATConv, to_hetero,Linear, HeteroConv, SAGEConv
from torch_geometric.nn.conv import HANConv
import torch.nn.functional as F
from torchmetrics import AUROC

class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_dim = 128, out_channels = 1):
        hidden_channels = 128
        out_channels = 1
        super(GraphNeuralNetwork, self).__init__()
#         self.lin = Linear(-1, 16)
        self.conv1 = SAGEConv((-1,-1), hidden_channels, normalize=True,aggr = "mean",root_weight = True)
        self.conv_end = SAGEConv(hidden_channels, out_channels, aggr = "mean", root_weight = True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv_end(x, edge_index)
        return x
    

In [ ]:
graph

In [ ]:
graph.validate()

## Shift data to device

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

graph = graph.to(device)

sepsis_cases = torch.count_nonzero(graph[PATIENT_NAME].y[train_mask])
control_cases = graph[PATIENT_NAME].y[train_mask].size(dim=0) - sepsis_cases
WEIGHT = (control_cases / (sepsis_cases + 1e-10))
WEIGHT = WEIGHT.to(device)

print("Data shifted to the device " + str(device))

In [ ]:
from torch_geometric.loader import NeighborLoader

kwargs = {
    "num_neighbors":[5] * 2,
    "batch_size":1_000_000
}
loader = NeighborLoader(
    graph,
    input_nodes=(PATIENT_NAME, graph[PATIENT_NAME].train_mask),
    **kwargs
)
val_loader = NeighborLoader(
    graph,
    input_nodes=(PATIENT_NAME, graph[PATIENT_NAME].val_mask),
    **kwargs
)

In [ ]:
for batch, graph in enumerate(loader):
    print(graph)
    break

## Model-Wrapper Class

In [ ]:
import torch 
from torch_geometric.nn import to_hetero
from tqdm.notebook import tqdm

class ModelWrapper():
    def __init__(self, graph):
        self.LEARNING_RATE = 3e-4
        self.MAX_EPOCHS = 10000
        
        self.graph = graph
#         model = HetGraphNeuralNetwork(graph.metadata(),graph.node_types, hidden_dim = 128, out_channels=1) 
        model = GraphNeuralNetwork(hidden_dim = 128, out_channels=1)         
        model = to_hetero(model, graph.metadata(), aggr='sum')
        self.model = model.to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LEARNING_RATE,betas=(0.9, 0.999), eps=1e-08)
        
        self.last_loss = 0
        self.increased_loss = 0
        self.BREAKING_THRESHOLD = 10    
        self.val_loss = []
        self.train_loss = []
    
    def validate(self):
        with torch.inference_mode():
            self.model.eval()
            acc_loss = 0
            batch_size = 0
            for batch, graph in enumerate(val_loader):
                out = self.model(graph.x_dict, graph.edge_index_dict)[PATIENT_NAME]
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out[val_mask]), graph[PATIENT_NAME].y[val_mask].type(torch.float32),
                                                          pos_weight=WEIGHT)
                acc_loss += loss.item()
                batch_size += 1
            avg_loss = acc_loss / batch_size
            self.val_loss.append(avg_loss)
            if avg_loss > self.last_loss:
                self.increased_loss += 1
            else:
                self.increased_loss = 0
            self.last_loss = avg_loss

    def train(self):
        for epoch in tqdm(range(self.MAX_EPOCHS)):
            acc_loss = 0
            batch_size = 0
            for batch, graph in enumerate(loader):
                self.model.train()
                self.optimizer.zero_grad()
                out = self.model(graph.x_dict, graph.edge_index_dict)[PATIENT_NAME]
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out[train_mask]), graph[PATIENT_NAME].y[train_mask].type(torch.float32),
                                                          pos_weight=WEIGHT)
                acc_loss += loss.item()
                batch_size += 1
            avg_loss = acc_loss / batch_size
            self.train_loss.append(avg_loss)
            loss.backward()
            self.optimizer.step()
            self.validate() 

            if self.increased_loss >= self.BREAKING_THRESHOLD:
                print(f"Breaked at {str(epoch)}")
                break
                
            
    def get_model(self):
        return self.model    
    
    def plot_loss(self):
        plt.plot(self.epochs, self.train_loss, 'g', label='Training loss')
        plt.plot(self.epochs, self.val_loss, 'y', label='Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [ ]:
model_wrapper = ModelWrapper(graph)
import time 
print("GIN-HET")
start = time.time()
model_wrapper.train()
print(time.time() - start )
model = model_wrapper.get_model()

In [ ]:
graph

## Evaluation

In [ ]:
def predict_proba(graph, mask):
    with torch.inference_mode():
        model.eval()
        logits = model(graph.x_dict, graph.edge_index_dict)[PATIENT_NAME]
        scores = torch.sigmoid(torch.squeeze(logits[mask]))
        scores = torch.unsqueeze(scores, 0)
        proba_predict = torch.concat((1- scores, scores), dim = 0)
    return torch.transpose(proba_predict, 0, 1)

def predict(graph, mask):
    pred = torch.round(predict_proba(graph, mask)[:, 1])
    return pred

model.predict_proba = predict_proba
model.predict = predict

In [ ]:
from dataAnalysis.Metrics import Evaluation

graph = graph.cpu()
model = model.cpu()
evaluation = Evaluation(y_test, y_gw_test, X_test, X_gw_test)
evaluation.set_test_args([graph, test_l_mask])
evaluation.set_gw_args([graph, test_gw_mask])

In [ ]:
evaluation.plot_confusion_matrix(model)
evaluation.get_df_metrics(model)

## Error evaluation

In [ ]:
import time

number_of_iterations = 100
dataframes = []
for i in range(number_of_iterations):
    graph = graph.to(device)
    
    
    start = time.time()
    model_wrapper = ModelWrapper(graph)
    model_wrapper.train()
    print(time.time() - start)
    
    def predict_proba(graph, mask):
        with torch.inference_mode():
            model.eval()
            logits = model(graph.x_dict, graph.edge_index_dict)[PATIENT_NAME]
            scores = torch.sigmoid(torch.squeeze(logits[mask]))
            scores = torch.unsqueeze(scores, 0)
            proba_predict = torch.concat((1- scores, scores), dim = 0)
        return torch.transpose(proba_predict, 0, 1)

    def predict(graph, mask):
        pred = torch.round(predict_proba(graph, mask)[:, 1])
        return pred
    model = model_wrapper.get_model()
    model.predict_proba = predict_proba
    model.predict = predict
    
    from dataAnalysis.Metrics import Evaluation

    graph = graph.cpu()
    model = model.cpu()
    evaluation = Evaluation(y_test, y_gw_test, X_test, X_gw_test)
    evaluation.set_test_args([graph, test_l_mask])
    evaluation.set_gw_args([graph, test_gw_mask])
    
    df = evaluation.get_df_metrics(model)
    dataframes.append(df)
for df in dataframes:
    print(df)